In [1]:
import os
import time
import math
import json
import redis
import asyncio
import traceback
from aredis import StrictRedis
from multiprocessing import Process

In [2]:
from pathlib import Path
os.chdir(Path(os.getcwd()).parent)

In [3]:
from libs.quotation import Quotation
from libs.dailydata import DailyData
from libs.utils import Utils
from libs.cython.compute import compute_stats

In [4]:
policy = asyncio.WindowsSelectorEventLoopPolicy()
asyncio.set_event_loop_policy(policy)

In [5]:
def assist(assist_idx, assist_count):
    
    ar = StrictRedis(host='127.0.0.1', port=6379, db=8)
    data = {}
    task_snapshotting = None
    
    def get_daily_data(date):
        if date not in data:
            dd = DailyData(date, create=False)
            data[date] = dd
        else:
            dd = data[date]
            
        return dd
    
        
    def compute_statistics(date):
        dd = get_daily_data(date)
        group_size = math.ceil(len(dd.symbols)/assist_count)
        scope = (
            assist_idx*group_size, 
            min((assist_idx+1)*group_size, len(dd.symbols))
        )
        basics = dd.basics[scope[0]:scope[1], :]
        for _, check_point in enumerate(daily_data.check_points):
            
            snapshot = dd.snapshots[_,scope[0]:scope[1], :]
            statistic = dd.statistic[_,scope[0]:scope[1], :]
            
            time_lapse = dd.get_time_lapse(_)
            ma5pm_anchor_idx = dd.get_ma5pm_anchor_idx(_)
            fs5p = dd.snapshots[ma5pm_anchor_idx,scope[0]:scope[1], :]
            
            compute_stats(snapshot, basics, statistic, fs5p, time_lapse)
            
    
    async def snapshotting(date):
        try:
            dd = get_daily_data(date)
            group_size = math.ceil(len(dd.symbols)/assist_count)
            scope = (
                assist_idx*group_size, 
                min((assist_idx+1)*group_size, len(dd.symbols))
            )

            q = Quotation(symbols=dd.symbols.tolist()[scope[0]:scope[1]])
            basics = dd.basics[scope[0]:scope[1], :]

            for _, check_point in enumerate(dd.check_points):
                if time.time() > check_point:
                    continue

                delay=(check_point-time.time())
                await asyncio.sleep(max(delay,0))
#                 await asyncio.sleep(5)

                try:
                    await q.snapshot(array=dd.snapshots[_,scope[0]:scope[1],:])

                    snapshot = dd.snapshots[_,scope[0]:scope[1], :]
                    statistic = dd.statistic[_,scope[0]:scope[1], :]

                    time_lapse = dd.get_time_lapse(_)
                    ma5pm_anchor_idx = dd.get_ma5pm_anchor_idx(_)
                    fs5p = dd.snapshots[ma5pm_anchor_idx,scope[0]:scope[1], :]

                    compute_stats(snapshot, basics, statistic, fs5p, time_lapse)

                    await ar.publish(f'hq_assist_{assist_idx}_snapshotting', json.dumps({"status":'successful',"idx":_,'check_point':int(check_point)}))
                        
                except Exception as e:
                    error = {
                        "status": 'failed',
                        "idx": _,
                        'check_point':int(check_point),
                        "exception": str(e),
                        'traceback': traceback.format_exc()
                    }
                    await ar.publish(f'hq_assist_{assist_idx}_snapshotting', json.dumps(error))
                    
#                 finally:
#                     if assist_idx == 0:
#                         dd.incremental_save(_)
                        
        finally:
            await q.exit()
    
    
    async def main():
        
        snapshotting_task = None
        
        while True:
            key, value = await ar.brpop(f'hq_assist_{assist_idx}')
            msg = json.loads(value)
            print(f'Assist[{assist_idx}] {msg}')
            
            if msg['command'] == 'snapshotting':
                snapshotting_task = asyncio.create_task(snapshotting(msg['date']))
                
            elif msg['command'] == 'compute_statistics':
                try:
                    compute_statistics(msg['date'])
                    await ar.lpush(f'hq_assist_{assist_idx}_compute_statistics', json.dumps({"status":'success'}))
                except Exception as e:
                    error = {
                        "status": 'failed',
                        "exception": str(e),
                        'traceback': traceback.format_exc()
                    }
                    await ar.lpush(f'hq_assist_{assist_idx}_compute_statistics', json.dumps(error))
                    
            elif msg['command'] == 'incremental_save':
                dd = get_daily_data(msg['date'])
                dd.incremental_save(msg['idx'])

            elif msg['command'] == 'quit':
                    
                if snapshotting_task and snapshotting_task.done() is False:
                    print(f'Assist[{assist_idx}]: snapshotting_task is going to be canceled')
                    snapshotting_task.cancel()
                    
                print(f'Assist[{assist_idx}]: sharedmemory is going to be closed')
                for date in data:
                    data[date].close_sharedmemory()
                break
                
            else:
                pass
            
#     asyncio.run(main())
    asyncio.create_task(main())
    
    return data
    
# if __name__ == '__main__':
#     rd = redis.Redis(host='127.0.0.1', port=6379, db=8)
#     for key in rd.keys():
#         rd.delete(key)

#     ####
#     Utils.update_symbols()
    
#     symbols = Utils.get_running_symbols()
#     assist_count = math.ceil(len(symbols)/800)+1
    
#     rd.set('hq_assist_count', assist_count)

#     processes = []
#     for _ in range(assist_count):
#         proc = Process(target=assist, args=(_, assist_count))
#         processes.append(proc)
#         proc.start()

#     for proc in processes:
#         proc.join()


In [6]:
await asyncio.sleep(int(time.mktime(time.strptime(f'{time.strftime("%Y-%m-%d")} 09:13:00', '%Y-%m-%d %H:%M:%S'))) - time.time()) 

rd = redis.Redis(host='127.0.0.1', port=6379, db=8)

symbols = Utils.get_running_symbols()
assist_count = math.ceil(len(symbols)/800)+1

rd.set('hq_assist_count', assist_count)

result = []
for _ in range(assist_count):
    data = assist(_, assist_count)
    result.append(data)

Assist[0] {'command': 'snapshotting', 'date': '20210623'}
Assist[1] {'command': 'snapshotting', 'date': '20210623'}
Assist[2] {'command': 'snapshotting', 'date': '20210623'}
Assist[6] {'command': 'snapshotting', 'date': '20210623'}
Assist[4] {'command': 'snapshotting', 'date': '20210623'}
Assist[3] {'command': 'snapshotting', 'date': '20210623'}
Assist[5] {'command': 'snapshotting', 'date': '20210623'}


In [ ]:
dd = DailyData('20210219')

In [ ]:
dd.close_sharedmemory()

In [ ]:
rd = redis.Redis(host='127.0.0.1', port=6379, db=8)

In [ ]:

# for key in rd.keys():
#     rd.delete(key)

In [ ]:
Utils.update_symbols()

In [ ]:
symbols = Utils.get_running_symbols()
assist_count = math.ceil(len(symbols)/800)+1
assist_count

In [ ]:
rd.set('hq_assist_count', assist_count)

In [ ]:
result = []
for _ in range(assist_count):
    data = assist(_, assist_count)
    result.append(data)

In [7]:
date = time.strftime('%Y%m%d')
result[0][date].get_snapshot('13:51:55')

,datetime,timestamp,name,open,close,now,high,low,turnover,volume,zhangfu,junjia,liangbi,zhangsu,tingban,ma5
symbol,,,,,,,,,,,,,,,,
000001,13:51:55,1624254715,平安银行,22.60,22.65,22.25,22.67,22.01,"46,434,392.00","1,036,948,572.87",-1.77,22.33,1.07,-1.55,NaN,22.90
000002,13:51:55,1624254715,万 科Ａ,24.50,24.57,24.12,24.53,24.11,"45,428,094.00","1,102,362,910.95",-1.83,24.27,0.91,-1.55,NaN,24.68
000004,13:51:55,1624254715,国华网安,19.50,19.78,20.21,20.99,19.18,"7,266,730.00","146,441,908.90",2.17,20.15,0.72,3.64,NaN,20.53
000005,13:51:55,1624254715,ST星源,1.81,1.82,1.81,1.83,1.81,"4,411,519.00","8,021,562.62",-0.55,1.82,0.97,0.00,NaN,1.83
000006,13:51:55,1624254715,深振业Ａ,5.20,5.24,5.23,5.26,5.20,"3,682,511.00","19,277,499.20",-0.19,5.23,0.93,0.58,NaN,5.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688777,13:51:55,1624254715,中控技术,89.65,89.69,91.90,92.78,87.19,"861,276.00","78,229,083.00",2.46,90.83,0.98,2.51,NaN,90.14
688788,13:51:55,1624254715,科思科技,132.16,132.14,136.30,139.39,132.00,"699,185.00","95,905,519.00",3.15,137.17,1.71,3.13,NaN,132.24
688819,13:51:55,1624254715,天能股份,40.72,40.86,41.95,42.30,40.42,"2,834,928.00","117,951,478.00",2.67,41.61,1.14,3.02,NaN,40.93


In [26]:
date = time.strftime('%Y%m%d')

In [14]:
securities = result[0][date].get_securities()

In [20]:
securities.loc['300057']

name            万顺新材
zt_price        6.16
dt_price        4.10
ma5vpm     42,081.33
mcap           26.61
sum4           20.75
sum9           46.31
sum19          95.39
sum29         141.62
sum59         281.96
Name: 300057, dtype: object

In [ ]:
############################## TEST #####################################

In [21]:
check_points = Utils.get_check_points()
symbols = Utils.get_running_symbols()

In [27]:
date = time.strftime('%Y%m%d')

In [28]:
data = result[0][date]

In [29]:
q = Quotation(symbols)
snapshot = await q.snapshot()

market_values = await q.get_market_values()

for _, symbol in enumerate(symbols):
    data.basics[_, 0] = market_values[symbol]['zt_price']
    data.basics[_, 1] = market_values[symbol]['dt_price']
    data.basics[_, 3] = market_values[symbol]['mcap']

await q.exit()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000015B7EECED30>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000015B7EECE1C0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000158F8ABE0A0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000158F8ABE130>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000015B7EEB3160>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000015B7EEB3670>


In [32]:
securities = data.get_securities()
securities

,name,zt_price,dt_price,ma5vpm,mcap,sum4,sum9,sum19,sum29,sum59
symbol,,,,,,,,,,
000001,平安银行,24.38,19.94,"242,040.40","4,457.50",91.17,211.14,454.66,690.02,"1,349.74"
000002,万 科Ａ,26.62,21.78,"278,420.50","2,354.56",98.73,226.40,493.45,762.96,"1,633.81"
000004,国华网安,22.85,18.69,"63,075.68",23.23,81.62,186.82,349.72,502.68,"1,028.56"
000005,ST星源,1.92,1.74,"25,987.79",19.89,7.33,16.73,34.76,53.17,123.03
000006,深振业Ａ,5.76,4.72,"21,713.14",70.87,20.81,46.74,99.45,153.09,314.56
...,...,...,...,...,...,...,...,...,...,...
688777,中控技术,110.40,73.60,"4,614.81",37.52,359.10,818.07,"1,729.69","2,537.90","5,002.13"
688788,科思科技,165.52,110.34,"2,493.83",23.26,529.56,"1,198.50","2,553.55","3,661.80","6,736.74"
688819,天能股份,50.54,33.70,"14,020.31",42.06,163.43,376.76,802.06,"1,208.27","2,568.15"


In [35]:
securities.loc['300057']

name            万顺新材
zt_price        6.41
dt_price        4.27
ma5vpm     42,081.33
mcap           31.94
sum4           20.75
sum9           46.31
sum19          95.39
sum29         141.62
sum59         281.96
Name: 300057, dtype: object

In [44]:
ss = data.get_snapshot('13:07:45')
ss

,datetime,timestamp,name,open,close,now,high,low,turnover,volume,zhangfu,junjia,liangbi,zhangsu,tingban,ma5
symbol,,,,,,,,,,,,,,,,
000001,13:07:45,1624338465,平安银行,22.38,22.16,22.73,22.77,22.25,"43,408,668.00","979,494,003.79",2.57,22.56,1.40,1.56,NaN,22.78
000002,13:07:45,1624338465,万 科Ａ,24.30,24.20,24.16,24.47,24.16,"34,376,457.00","835,808,896.75",-0.17,24.31,0.96,-0.58,NaN,24.58
000004,13:07:45,1624338465,国华网安,20.30,20.77,20.56,20.56,20.02,"4,443,189.00","90,042,699.95",-1.01,20.27,0.55,1.28,NaN,20.44
000005,13:07:45,1624338465,ST星源,1.82,1.83,1.88,1.89,1.81,"6,709,009.00","12,380,126.08",2.73,1.85,2.02,3.30,NaN,1.84
000006,13:07:45,1624338465,深振业Ａ,5.24,5.24,5.24,5.27,5.22,"3,270,300.00","17,174,296.00",0.00,5.25,1.18,0.00,NaN,5.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688777,13:07:45,1624338465,中控技术,91.80,92.00,91.44,92.81,89.60,"571,489.00","51,843,772.00",-0.61,90.72,0.97,-0.39,NaN,90.11
688788,13:07:45,1624338465,科思科技,139.20,137.93,135.79,139.20,133.79,"584,001.00","79,228,813.00",-1.55,135.67,1.83,-2.45,NaN,133.07
688819,13:07:45,1624338465,天能股份,42.12,42.12,41.90,42.27,41.65,"1,237,633.00","51,811,621.00",-0.52,41.86,0.69,-0.52,NaN,41.07


In [45]:
ss.loc['600354']

datetime          13:07:45
timestamp       1624338465
name                  敦煌种业
open                  5.80
close                 5.76
now                   6.27
high                  6.34
low                   5.72
turnover     42,527,873.00
volume      262,371,856.00
zhangfu               8.85
junjia                6.17
liangbi               2.27
zhangsu               8.10
tingban               1.00
ma5                   5.74
Name: 600354, dtype: object

In [46]:
st = time.time()
for _ in range(len(data.check_points)):
    time_lapse = data.get_time_lapse(_)
    ma5pm_anchor_idx = data.get_ma5pm_anchor_idx(_)
    fs5p = data.snapshots[ma5pm_anchor_idx]
    compute_stats(data.snapshots[_], data.basics, data.statistic[_], fs5p, time_lapse)
et = time.time()
et - st

1.097827672958374

In [47]:
data.save()

文件 [ D:\workspace\python\Securities\storage\20210622.hdf5 ] 已经存在，将被删除 ... ... 已被删除
行情 已经 写入文件： D:\workspace\python\Securities\storage\20210622.hdf5
